In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Retail Product Customization with Imagen and Veo

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/retail_with_veo_and_imagen.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fvision%2Fuse-cases%2Fretail_with_veo_and_imagen.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/vision/use-cases/retail_with_veo_and_imagen.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/retail_with_veo_and_imagen.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/retail_with_veo_and_imagen.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/retail_with_veo_and_imagen.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/retail_with_veo_and_imagen.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/53/X_logo_2023_original.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/retail_with_veo_and_imagen.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/vision/use-cases/retail_with_veo_and_imagen.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
|Author(s) | [Katie Nguyen](https://github.com/katiemn) |

## Overview

### Veo 2

Veo 2 on Vertex AI brings Google's state of the art video generation capabilities to application developers. It's capable of creating videos with astonishing detail that simulate real-world physics across a wide range of visual styles.

In this tutorial, you will walk through a retail use case that illustrates how you can use Veo and Imagen to:
- Start with existing prodcut images
- Edit product photo backgrounds
- Add motion to create assets that can be used for product catalogues, recommendation engines, or advertisements

You will utilize the Google Gen AI SDK for Python to interact with Veo 2 and Imagen 3 to generate new product images and videos.

## Get started

### Install Google Gen AI SDK for Python and other libraries

In [ ]:
%pip install --upgrade --quiet google-genai
%pip install -q mediapy

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Import libraries

In [3]:
import time
import urllib

from PIL import Image as PIL_Image
from google import genai
from google.genai import types
import matplotlib.pyplot as plt
import mediapy as media

### Set Google Cloud project information and create client

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Define helper functions to display media

In [5]:
def display_images(image) -> None:
    fig, axis = plt.subplots(1, 1, figsize=(12, 6))
    axis.imshow(image)
    axis.set_title("Product Image")
    axis.axis("off")
    plt.show()


def show_video(gcs_uri):
    file_name = gcs_uri.split("/")[-1]
    !gsutil cp {gcs_uri} {file_name}
    media.show_video(media.read_video(file_name), height=500)

### Load the generative media models

In [6]:
video_model = "veo-2.0-generate-001"
edit_model = "imagen-3.0-capability-001"

### Load the original product image

Start by loading the initial product image. In this case, you'll be loading an image of a suitcase saved in Google Cloud Storage.


In [ ]:
product_image = types.Image(
    gcs_uri="gs://cloud-samples-data/generative-ai/image/suitcase.png"
)

product_image_show = PIL_Image.open(
    urllib.request.urlopen(
        "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/suitcase.png"
    )
)

display_images(product_image_show)

### Edit the product background

Next, create a background mask and supply a prompt to guide background replacement of the product image. To maintain the original product set the edit_mode to "EDIT_MODE_BGSWAP".

Finally, save the edited image locally to reference it later in your video generation prompt.

In [ ]:
raw_ref_image = types.RawReferenceImage(reference_image=product_image, reference_id=0)
mask_ref_image = types.MaskReferenceImage(
    reference_id=1,
    reference_image=None,
    config=types.MaskReferenceConfig(mask_mode="MASK_MODE_BACKGROUND"),
)

prompt = "a busy airport"
edited_image = client.models.edit_image(
    model=edit_model,
    prompt=prompt,
    reference_images=[raw_ref_image, mask_ref_image],
    config=types.EditImageConfig(
        edit_mode="EDIT_MODE_BGSWAP",
        number_of_images=1,
        seed=1,
        safety_filter_level="BLOCK_MEDIUM_AND_ABOVE",
        person_generation="ALLOW_ADULT",
    ),
)

edited_product_image = edited_image.generated_images[0].image
display_images(edited_product_image._pil_image)
edited_product_image.save("edited_product_image.png")

### Generate videos from an image input

Now, you'll take the edited image and add motion with Veo. In order to generate a video in the following sample, specify the following info:
- **File location:** The generated video will be shown below with support from a previously defined helper function. The video will also be stored in Cloud Storage once video generation is complete. Specify the file path where you'd like this video to be stored in the output_gcs field.


In [ ]:
output_gcs = "gs://"  # @param {type: 'string'}
img = types.Image.from_file(location="edited_product_image.png")

operation = client.models.generate_videos(
    model=video_model,
    image=types.Image(
        image_bytes=img.image_bytes,
        mime_type="image/png",
    ),
    config=types.GenerateVideosConfig(
        output_gcs_uri=output_gcs,
        aspect_ratio="9:16",
        number_of_videos=1,
        person_generation="allow_adult",
    ),
)

while not operation.done:
    time.sleep(15)
    operation = client.operations.get(operation)
    print(operation)

if operation.response:
    show_video(operation.result.generated_videos[0].video.uri)